In [1]:
import pandas as pd
from datetime import date
today = date.today()
from cafu.queries.partida import Partida
from cafu.metadata import path
path_datalake = path('datalake')

In [2]:
path_camp = path_datalake+'/jogos_ids/franca/2019-2020.csv'
df = pd.read_csv(path_camp)

In [3]:
df

,jogo_id,dates,rodada,time_casa,time_visitante
0,541855,2020-05-23,38,Stade Rennais,AS Mônaco
1,541856,2020-05-23,38,Metz,Angers
2,541857,2020-05-23,38,Lyon,Brest
3,541858,2020-05-23,38,Saint-Etienne,Dijon FCO
4,541859,2020-05-23,38,Bordeaux,Lille
...,...,...,...,...,...
375,542230,2019-08-10,1,Nice,SC Amiens
376,542231,2019-08-10,1,Dijon FCO,Saint-Etienne
377,542232,2019-08-10,1,Montpellier,Stade Rennais
378,542233,2019-08-10,1,Marseille,Stade de Reims


In [4]:
jogo_id = 541855

In [5]:
req = Partida(jogo_id, check_status=False)

campeonato = req.campeonato() # Retorna o campeonato.
data = req.day() # Retorna a data.
nomes_times = req.nomes_times() # Retorna o nome dos times.
formacao = req.formacao() # Retorna as formações dos times.
jogadores = req.jogadores() # Retorna as escalações, e substituições.
gols = req.gols() # Retorna os gols que ocorreram, autor do gol e tempo. 
placar = req.placar() # Retorna o placar. 
posse = req.posse() # Retorna a posse de bola.
chutes_fora_nogol = req.chutes_fora_nogol() # Retorna a quantidade de chutes, no gol e fora do gol.
faltas = req.faltas() # Retorna a quantidade de faltas cometidas.
cartoes_amarelos = req.cartoes_amarelos() # Retorna a quantidade de cartões amarelos.
cartoes_vermelhos = req.cartoes_vermelhos() # Retorna a quantidade de cartões vermelhos.
impedimentos = req.impedimentos() # Retorna a quantidade de impedimentos.
escanteios = req.escanteios() # Retorna a quantidade de escanteios.
defesas = req.defesas() 
minuto_a_minuto = req.minuto_a_minuto() # Retorna a descrição do jogo, minuto a minuto.

## Visualizando informações

In [6]:
campeonato

'Temporada regular'

In [7]:
data

datetime.datetime(2020, 5, 23, 0, 0)

In [8]:
nomes_times

('Stade Rennais', 'AS Mônaco')

In [9]:
formacao

In [10]:
jogadores

In [11]:
gols

(None, None)

In [12]:
placar

In [13]:
posse

In [14]:
chutes_fora_nogol

In [15]:
faltas

In [16]:
cartoes_amarelos

In [17]:
cartoes_vermelhos

In [18]:
impedimentos

In [19]:
escanteios

In [20]:
defesas

In [21]:
minuto_a_minuto

[]

In [22]:
if (
    formacao is None and
    defesas is None and
    jogadores is None and
    placar is None and
    posse is None and
    chutes_fora_nogol is None and
    faltas is None and
    cartoes_amarelos is None and
    cartoes_vermelhos is None and
    impedimentos is None and
    escanteios is None and
    defesas is None and
    len(minuto_a_minuto) == 0 and
    str(data)<str(today)
   ): 
    req = Partida(jogo_id)
    print(req.status)

Cancelado


## --------------

In [1]:
from cafu.etl import update_partidas
from cafu.utils import get_spark
spark = get_spark(1)

22/03/16 13:13:33 WARN Utils: Your hostname, luan-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/03/16 13:13:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/luan/.ivy2/cache
The jars for the packages stored in: /home/luan/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/spark/spark-3.0.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4fa68d93-88ff-49af-ade7-c4d07398e200;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in cen

In [2]:
from cafu.utils.etl.datalake import partidas_desatualizadas
from cafu.queries.partida import Partida
from cafu.utils import get_spark
from cafu.metadata import get_schema
from cafu.metadata import campeonato_espn
from datetime import datetime
import pyspark.sql.functions as F

In [3]:
jogo_id = 569662
req = Partida(jogo_id, check_status = False)

In [4]:
campeonato = req.campeonato()
date = req.day()
time_casa, time_visitante = req.nomes_times()
formacao_time_casa, formacao_time_visitante = req.formacao()
time_casa_gols_feitos, time_visitante_gols_feitos = req.placar()
time_casa_posse, time_visitante_posse = req.posse()
chutes_fora_nogol = req.chutes_fora_nogol()
try:
    time_casa_faltas_cometidas, time_visitante_faltas_cometidas = req.faltas()
except:
    time_casa_faltas_cometidas, time_visitante_faltas_cometidas = None, None
time_casa_cartoes_amarelos, time_visitante_cartoes_amarelos = req.cartoes_amarelos()
time_casa_cartoes_vermelhos, time_visitante_cartoes_vermelhos = req.cartoes_vermelhos()
time_casa_impedimentos, time_visitante_impedimentos = req.impedimentos()
time_casa_escanteios, time_visitante_escanteios = req.escanteios()
try:
    time_casa_defesas, time_visitante_defesas = req.defesas()
except:
    time_casa_defesas, time_visitante_defesas = None, None

data = [
        {
         'jogo_id': jogo_id,   
         'campeonato': campeonato,
         'campeonato_metadata': '1',
         'temporada_metadata': '1',
         'rodada': 1,   
         'date': date,
         'time_casa': time_casa,
         'time_visitante': time_visitante,
         'formacao_time_casa': formacao_time_casa,
         'formacao_time_visitante': formacao_time_visitante,
         'time_casa_gols_feitos': time_casa_gols_feitos,
         'time_visitante_gols_feitos': time_visitante_gols_feitos,
         'time_casa_posse': time_casa_posse,
         'time_visitante_posse': time_visitante_posse,
         'time_casa_chutes_fora': chutes_fora_nogol[0][0],
         'time_casa_chutes_nogol': chutes_fora_nogol[0][1],
         'time_visitante_chutes_fora': chutes_fora_nogol[1][0],
         'time_visitante_chutes_nogol': chutes_fora_nogol[1][1],
         'time_casa_faltas_cometidas': time_casa_faltas_cometidas,
         'time_visitante_faltas_cometidas': time_visitante_faltas_cometidas,
         'time_casa_cartoes_amarelos': time_casa_cartoes_amarelos,
         'time_visitante_cartoes_amarelos': time_visitante_cartoes_amarelos,
         'time_casa_cartoes_vermelhos': time_casa_cartoes_vermelhos,
         'time_visitante_cartoes_vermelhos': time_visitante_cartoes_vermelhos,
         'time_casa_impedimentos': time_casa_impedimentos,
         'time_visitante_impedimentos': time_visitante_impedimentos,   
         'time_casa_escanteios': time_casa_escanteios,   
         'time_visitante_escanteios': time_visitante_escanteios,   
         'time_casa_defesas': time_casa_defesas,   
         'time_visitante_defesas': time_visitante_defesas,   
         'date_update':  datetime.now()
        }
       ]

schema = get_schema('partidas_resumo')
df_resumo = spark.createDataFrame(data, schema=schema)

In [5]:
df_resumo.toPandas()

,campeonato,campeonato_metadata,temporada_metadata,date,formacao_time_casa,formacao_time_visitante,jogo_id,time_casa,time_casa_cartoes_amarelos,time_casa_cartoes_vermelhos,...,time_visitante_cartoes_vermelhos,time_visitante_chutes_fora,time_visitante_chutes_nogol,time_visitante_defesas,time_visitante_escanteios,time_visitante_faltas_cometidas,time_visitante_gols_feitos,time_visitante_impedimentos,time_visitante_posse,date_update
0,2020 Brasileiro Serie A,1,1,2021-02-02,None,None,569662,Palmeiras,2,0,...,0,13,3,0,3,8,1,0,0.46,2022-03-16 13:14:12.991572


In [ ]:
df_resumo.show()

In [6]:
jogadores = req.jogadores()
gols = req.gols()
minuto_a_minuto = req.minuto_a_minuto()

In [15]:
req.jogadores()

(None, None)

In [7]:
jogadores_casa = (
                     spark.createDataFrame(jogadores[0])
                     .withColumn('casa_visitante', F.lit('casa'))
                     .withColumn('jogo_id', F.lit(jogo_id))
                 )
jogadores_visitante = (
                          spark.createDataFrame(jogadores[1])
                          .withColumn('casa_visitante', F.lit('visitante'))
                          .withColumn('jogo_id', F.lit(jogo_id))
                      )
df_jogadores = (
                   jogadores_casa
                   .unionByName(jogadores_visitante)
                   .withColumn('date_update', F.lit(datetime.now()))
               )

schema = get_schema('partidas_jogadores_minutagens')
df_jogadores = spark.createDataFrame(df_jogadores.collect(), schema=schema)

TypeError: 'NoneType' object is not iterable

In [8]:
df_jogadores.show()

NameError: name 'df_jogadores' is not defined

In [11]:
if gols[0] is not None:
    gols_casa = (
                         spark.createDataFrame(gols[0])
                         .withColumn('casa_visitante', F.lit('casa'))
                         .withColumn('jogo_id', F.lit(jogo_id))
                     )
if gols[1] is not None:
    gols_visitante = (
                              spark.createDataFrame(gols[1])
                              .withColumn('casa_visitante', F.lit('visitante'))
                              .withColumn('jogo_id', F.lit(jogo_id))
                          )
if (gols[0] is not None) and (gols[1] is not None):
    df_gols = (
                  gols_casa
                  .unionByName(gols_visitante)
                  .withColumn('minuto_gol', F.explode('minutos_gols'))
                  .drop('minutos_gols')
                  .withColumn('date_update', F.lit(datetime.now()))
              )
elif gols[0] is not None:
    df_gols = (
                  gols_casa
                  .withColumn('minuto_gol', F.explode('minutos_gols'))
                  .drop('minutos_gols')
                  .withColumn('date_update', F.lit(datetime.now()))
              )
elif gols[1] is not None:
    df_gols = (
                  gols_visitante
                  .withColumn('minuto_gol', F.explode('minutos_gols'))
                  .drop('minutos_gols')
                  .withColumn('date_update', F.lit(datetime.now()))
              )
    
if (gols[0] is not None) or (gols[1] is not None):
    schema = get_schema('partidas_gols')
    df_gols = spark.createDataFrame(df_gols.collect(), schema=schema)
else:
    df_gols = None

/usr/local/spark/spark-3.0.3-bin-hadoop2.7/python/pyspark/sql/session.py:381: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [12]:
df_gols.show()

+--------------+--------------+-------+----------+--------------------+
|       jogador|casa_visitante|jogo_id|minuto_gol|         date_update|
+--------------+--------------+-------+----------+--------------------+
|Emerson Santos|          casa| 569662|        16|2022-03-16 13:16:...|
|Rafael Navarro|     visitante| 569662|        60|2022-03-16 13:16:...|
+--------------+--------------+-------+----------+--------------------+



In [13]:
df_minuto_a_minuto = (
                         spark.createDataFrame(minuto_a_minuto)
                         .withColumnRenamed('descrição', 'descricao')
                         .withColumn('jogo_id', F.lit(jogo_id))
                         .withColumn('date_update', F.lit(datetime.now()))
                     )

schema = get_schema('partidas_descricoes')
df_minuto_a_minuto = spark.createDataFrame(df_minuto_a_minuto.collect(), schema=schema)

In [14]:
df_minuto_a_minuto.show()

+--------------------+-----+-------+--------------------+
|           descricao|tempo|jogo_id|         date_update|
+--------------------+-----+-------+--------------------+
|Fim do Jogo, Palm...|   90| 569662|2022-03-16 13:16:...|
|Fim do segundo te...|   90| 569662|2022-03-16 13:16:...|
|Substituição Bota...|   90| 569662|2022-03-16 13:16:...|
|Substituição Bota...|   90| 569662|2022-03-16 13:16:...|
|Escanteio, Palmei...|   89| 569662|2022-03-16 13:16:...|
|Escanteio, Palmei...|   88| 569662|2022-03-16 13:16:...|
|Escanteio, Palmei...|   88| 569662|2022-03-16 13:16:...|
|Substituição Palm...|   87| 569662|2022-03-16 13:16:...|
|Impedimento, Palm...|   86| 569662|2022-03-16 13:16:...|
|Gustavo Scarpa (P...|   86| 569662|2022-03-16 13:16:...|
|Falta cometida po...|   86| 569662|2022-03-16 13:16:...|
|Oportunidade perd...|   85| 569662|2022-03-16 13:16:...|
|Escanteio, Botafo...|   85| 569662|2022-03-16 13:16:...|
|Willian (Palmeira...|   84| 569662|2022-03-16 13:16:...|
|Falta cometid

In [20]:
from pyspark.sql.types import (StructType, StructField, 
                               StringType, DateType, TimestampType, 
                               IntegerType, FloatType)

schema = StructType([ 
                     StructField("jogo_id",IntegerType(),True), 
                     StructField("campeonato",StringType(),True), 
                     StructField("campeonato_metadata",StringType(),True), 
                     StructField("temporada_metadata",StringType(),True), 
                     StructField("rodada",IntegerType(),True),
                     StructField("date_update", TimestampType(), True)
                   ])

data = [{
        'jogo_id': jogo_id,   
        'campeonato': campeonato,
        'campeonato_metadata': '1',
        'temporada_metadata': '1',
        'rodada': 1,
        'date_update': datetime.now()
       }]
spark.createDataFrame(data, schema = schema).show()

+-------+--------------------+-------------------+------------------+------+--------------------+
|jogo_id|          campeonato|campeonato_metadata|temporada_metadata|rodada|         date_update|
+-------+--------------------+-------------------+------------------+------+--------------------+
| 614698|2021-22 Italian S...|                  1|                 1|     1|2022-03-15 15:30:...|
+-------+--------------------+-------------------+------------------+------+--------------------+



In [13]:
df_jogadores.show()

+------+-------------+--------------+--------------------+--------------+-------+--------------------+
|camisa|final_jogando|inicio_jogando|                nome|casa_visitante|jogo_id|         date_update|
+------+-------------+--------------+--------------------+--------------+-------+--------------------+
|    31|           90|             0|     Boris Radunovic|          casa| 614698|2022-03-15 15:03:...|
|     3|           86|             0|   Edoardo Goldaniga|          casa| 614698|2022-03-15 15:03:...|
|    23|           90|            86|     Luca Ceppitelli|          casa| 614698|2022-03-15 15:03:...|
|    33|           90|             0|          Adam Obert|          casa| 614698|2022-03-15 15:03:...|
|    15|           45|             0|      Giorgio Altare|          casa| 614698|2022-03-15 15:03:...|
|    66|           90|            45|       Matteo Lovato|          casa| 614698|2022-03-15 15:03:...|
|    29|           81|             0|             Dalbert|          casa|

In [60]:
now

datetime.datetime(2022, 3, 15, 14, 40, 31, 979178)

In [46]:
df = spark.createDataFrame(data=[],schema=schema)

In [49]:
df.write.parquet('teste')

In [58]:
df1 = spark.read.parquet('teste')

In [59]:
df1.toPandas()

,campeonato,date,formacao_time_casa,formacao_time_visitante,jogo_id,rodada,time_casa,time_casa_cartoes_amarelos,time_casa_cartoes_vermelhos,time_casa_chutes_fora,...,time_visitante_cartoes_amarelos,time_visitante_cartoes_vermelhos,time_visitante_chutes_fora,time_visitante_chutes_nogol,time_visitante_defesas,time_visitante_escanteios,time_visitante_faltas_cometidas,time_visitante_gols_feitos,time_visitante_impedimentos,time_visitante_posse
0,2021-22 Italian Serie A,2022-01-23,3-5-2,4-3-3,614698,1,Cagliari,3,0,9,...,3,1,10,6,1,4,16,1,1,0.6
1,2021-22 Italian Serie A,2022-01-23,3-5-2,4-3-3,614698,1,Cagliari,3,0,9,...,3,1,10,6,1,4,16,1,1,0.6


In [ ]:
df_resumo

In [44]:
df_resumo.printSchema()

root
 |-- campeonato: string (nullable = true)
 |-- date: date (nullable = true)
 |-- formacao_time_casa: string (nullable = true)
 |-- formacao_time_visitante: string (nullable = true)
 |-- jogo_id: integer (nullable = true)
 |-- rodada: integer (nullable = true)
 |-- time_casa: string (nullable = true)
 |-- time_casa_cartoes_amarelos: integer (nullable = true)
 |-- time_casa_cartoes_vermelhos: integer (nullable = true)
 |-- time_casa_chutes_fora: integer (nullable = true)
 |-- time_casa_chutes_nogol: integer (nullable = true)
 |-- time_casa_defesas: integer (nullable = true)
 |-- time_casa_escanteios: integer (nullable = true)
 |-- time_casa_faltas_cometidas: integer (nullable = true)
 |-- time_casa_gols_feitos: integer (nullable = true)
 |-- time_casa_impedimentos: integer (nullable = true)
 |-- time_casa_posse: float (nullable = true)
 |-- time_visitante: string (nullable = true)
 |-- time_visitante_cartoes_amarelos: integer (nullable = true)
 |-- time_visitante_cartoes_vermelhos: 

In [18]:
req.placar()

(1, 1)

In [14]:
spark.createDataFrame(data).show()

22/03/11 16:31:11 ERROR Executor: Exception in task 0.0 in stage 1.0 (TID 1)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/luan/anaconda3/envs/futebol/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 473, in main
    raise Exception(("Python in worker has different version %s than that in " +
Exception: Python in worker has different version 3.9 than that in driver 3.8, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:517)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:652)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:635)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:470)
	at org.apache.spark.Interru

Py4JJavaError: An error occurred while calling o176.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (10.0.2.15 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/luan/anaconda3/envs/futebol/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 473, in main
    raise Exception(("Python in worker has different version %s than that in " +
Exception: Python in worker has different version 3.9 than that in driver 3.8, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:517)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:652)
	at org.apache.spark.api.pyth

In [27]:
p = partidas_desatualizadas()

In [33]:
p['brasil']['2021-2021'][0]

598225

In [31]:
for c in p:
    for t in p[c]:
        print(c, t)

brasil 2021-2021
brasil 2020-2020
brasil 2019-2019
brasil 2018-2018
brasil 2017-2017
brasil 2016-2016
brasil 2015-2015
brasil 2014-2014
espanha 2021-2022
espanha 2020-2021
espanha 2019-2020
espanha 2018-2019
espanha 2017-2018
espanha 2016-2017
espanha 2015-2016
espanha 2014-2015
espanha 2013-2014
espanha 2012-2013
espanha 2011-2012
franca 2021-2022
franca 2020-2021
franca 2019-2020


In [29]:
p['brasil']

{'2021-2021': [598225,
  598226,
  598227,
  598228,
  598229,
  598230,
  598231,
  598232,
  598233,
  598234,
  598235,
  598236,
  598237,
  598238,
  598239,
  598240,
  598241,
  598242,
  598243,
  598244,
  598245,
  598246,
  598247,
  598248,
  598249,
  598250,
  598251,
  598252,
  598253,
  598254,
  598255,
  598256,
  598257,
  598258,
  598259,
  598260,
  598261,
  598262,
  598263,
  598264,
  598265,
  598266,
  598267,
  598268,
  598269,
  598270,
  598271,
  598272,
  598273,
  598274,
  598275,
  598276,
  598277,
  598278,
  598279,
  598280,
  598281,
  598282,
  598283,
  598284,
  598285,
  598286,
  598287,
  598288,
  598289,
  598290,
  598291,
  598292,
  598293,
  598294,
  598295,
  598296,
  598297,
  598298,
  598299,
  598300,
  598301,
  598302,
  598303,
  598304,
  598305,
  598306,
  598307,
  598308,
  598309,
  598310,
  598311,
  598312,
  598313,
  598314,
  598315,
  598316,
  598317,
  598318,
  598319,
  598320,
  598321,
  598322,
  59832